In [1]:
import pandas as pd
import numpy as np
import random 
import numpy.random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import scipy.special







In [2]:
orginal_data = pd.read_csv("F:\\abc.csv")
mu_eTIV, sigma_eTIV = orginal_data['eTIV'].mean(), orginal_data['eTIV'].var()
mu_MMSE, sigma_MMSE = orginal_data['MMSE'].mean(), orginal_data['MMSE'].var()
mu_EDUC, sigma_EDUC = orginal_data['EDUC'].mean(), orginal_data['EDUC'].var()
age_range = (60, 100)

def build_lstm_rnn(input_shape):
    lstm_rnn = tf.keras.Sequential()
    lstm_rnn.add(tf.keras.layers.LSTM(100, return_sequences=True, input_shape=input_shape))
    lstm_rnn.add(tf.keras.layers.LSTM(50, return_sequences=False))
    lstm_rnn.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    lstm_rnn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    return lstm_rnn

accuracy_list = []
recall_list = []
precision_list = []

In [3]:
for _ in range(5):
    N = 100
    m = 5

    Y = eta = np.zeros((N, m))
    Age = np.random.choice(range(age_range[0], age_range[1] + 1), size=N)
    MMSE = np.round(np.random.normal(mu_MMSE, sigma_MMSE, size=N))
    EDUC = np.round(np.random.normal(mu_EDUC, sigma_EDUC, size=N))
    Sex = np.random.binomial(1, 0.2, size=N)
    eTIV = np.round(np.random.normal(mu_eTIV, sigma_eTIV, size=N))
    ASF = np.random.normal(size=N)
    nWBV = np.random.normal(size=N)
    Visit = np.round(np.linspace(0, 4, num=m))

    B = np.array([0.1, -0.1, 0.2, -0.2, 0.3, 0.3, 0.1, 0.1, 0.2, 0.1])
    b = np.random.normal(0, 0.5, size=N)

    for i in range(N):
        for j in range(m):
            eta[i, j] = B[0] + B[1] * Visit[j] + B[2] * Age[i] + B[3] * MMSE[i] + B[4] * EDUC[i] + B[5] * Sex[i] + \
                        B[6] * eTIV[i] + B[7] * ASF[i] + B[8] * nWBV[i] + b[i]
            p = scipy.special.expit(eta[i, j])
            Y[i, j] = np.random.binomial(1, p)

   

    mat = np.empty((N * m, 10))
    p = 0
    for i in range(N):
        jk = 0
        for jj in range(p, p + m):
            mat[jj, 0] = i + 1
            mat[jj, 1] = Visit[jk]
            mat[jj, 2] = Age[i]
            mat[jj, 3] = MMSE[i]
            mat[jj, 4] = EDUC[i]
            mat[jj, 5] = Sex[i]
            mat[jj, 6] = eTIV[jk]
            mat[jj, 7] = ASF[jk]
            mat[jj, 8] = nWBV[jk]
            mat[jj, 9] = Y[i, jk]
            jk += 1
        p += m

    # Convert the `mat` array to a DataFrame
    df_mat = pd.DataFrame(mat, columns=['ID', 'Visit', 'Age', 'MMSE', 'EDUC', 'Sex', 'eTIV', 'ASF', 'nWBV', 'Y'])
    df = pd.get_dummies(df_mat, columns=["Visit"], prefix="Visit")
    RANDOM_STATE = 13
    splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state=RANDOM_STATE)
    split = splitter.split(df, groups=df['ID'])
    train_indexes, test_indexes = next(split)

    X_train = df.iloc[train_indexes]
    X_test = df.iloc[test_indexes]
    age_scaler = MinMaxScaler()
    educ_scaler = MinMaxScaler()
    mmse_scaler = MinMaxScaler()
    etiv_scaler = MinMaxScaler()

    age_scaler.fit(X_train["Age"].to_numpy().reshape(-1, 1))
    educ_scaler.fit(X_train["EDUC"].to_numpy().reshape(-1, 1))
    mmse_scaler.fit(X_train["MMSE"].to_numpy().reshape(-1, 1))
    etiv_scaler.fit(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_train["Age"] = age_scaler.transform(X_train["Age"].to_numpy().reshape(-1, 1))
    X_train["EDUC"] = educ_scaler.transform(X_train["EDUC"].to_numpy().reshape(-1, 1))
    X_train["MMSE"] = mmse_scaler.transform(X_train["MMSE"].to_numpy().reshape(-1, 1))
    X_train["eTIV"] = etiv_scaler.transform(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_test["Age"] = age_scaler.transform(X_test["Age"].to_numpy().reshape(-1, 1))
    X_test["EDUC"] = educ_scaler.transform(X_test["EDUC"].to_numpy().reshape(-1, 1))
    X_test["MMSE"] = mmse_scaler.transform(X_test["MMSE"].to_numpy().reshape(-1, 1))
    X_test["eTIV"] = mmse_scaler.transform(X_test["eTIV"].to_numpy().reshape(-1, 1))
    visit_3 = df[df["Visit_4.0"] == 1][["ID", "Y"]]
    y_train_final = []
    for id in X_train["ID"]:
        y_train_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_test_final = []
    for id in X_test["ID"]:
        y_test_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_train_super_final = pd.DataFrame({
    "Group": y_train_final})
    y_test_super_final = pd.DataFrame({
    "Group": y_test_final})
    X_test_final = X_test.drop(["ID"], axis = 1)
    X_train_final = X_train.drop(["ID"], axis = 1)
    rows_n = 5

    X_train_super_final = np.reshape(X_train_final.to_numpy(),(X_train_final.shape[0]//5,X_train_final.shape[1],rows_n))
    X_test_super_final = np.reshape(X_test_final.to_numpy(),(X_test_final.shape[0]//5,X_train_final.shape[1],rows_n))
    lstm_rnn = build_lstm_rnn((X_train_super_final.shape[1],X_train_super_final.shape[2]) )
    lstm_rnn.fit(X_train_super_final, y_train_super_final, epochs = 100)
    not_final_lstm_rnn_prediction = lstm_rnn.predict(X_test_super_final)
    lstm_rnn_prediction = np.where(not_final_lstm_rnn_prediction > 0.5, 1, 0)
    lstm_rnn_conf = confusion_matrix(y_test_super_final,lstm_rnn_prediction)
    lstm_rnn_plot_conf = ConfusionMatrixDisplay(lstm_rnn_conf)
    lstm_rnn_plot_conf.plot()
    TP=lstm_rnn_conf[1,1]
    FP=lstm_rnn_conf[0,1]
    TN=lstm_rnn_conf[0,0]
    FN=lstm_rnn_conf[1,0]
    accuracy=(TP+TN)/(TP+FP+FN+TN)
    recall =TP/(TP+FN)
    precision =TP/(TP+TN)
    accuracy_list.append(accuracy)
    recall_list.append(recall)
    precision_list.append(precision)

print("Accuracy List:", accuracy_list)
print("Recall List:", recall_list)
print("Precision List:", precision_list)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18224\2871003694.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["Age"] = age_scaler.transform(X_train["Age"].to_numpy().reshape(-1, 1))
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18224\2871003694.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["EDUC"] = educ_scaler.transform(X_train["EDUC"].to_numpy().reshape(-1, 1))
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18224\2871003694.py:67: SettingWithCopyWarning: 
A value is trying to be s

ValueError: Data cardinality is ambiguous:
  x sizes: 80
  y sizes: 400
Make sure all arrays contain the same number of samples.

In [20]:
for _ in range(5):
    N = 100
    m = 5

    Y = eta = np.zeros((N, m))
    Age = np.random.choice(range(age_range[0], age_range[1] + 1), size=N)
    MMSE = np.round(np.random.normal(mu_MMSE, sigma_MMSE, size=N))
    EDUC = np.round(np.random.normal(mu_EDUC, sigma_EDUC, size=N))
    Sex = np.random.binomial(1, 0.2, size=N)
    eTIV = np.round(np.random.normal(mu_eTIV, sigma_eTIV, size=N))
    ASF = np.random.normal(size=N)
    nWBV = np.random.normal(size=N)
    Visit = np.round(np.linspace(0, 4, num=m))

    B = np.array([0.1, -0.1, 0.2, -0.2, 0.3, 0.3, 0.1, 0.1, 0.2, 0.1])
    b = np.random.normal(0, 0.5, size=N)

    for i in range(N):
        for j in range(m):
            eta[i, j] = B[0] + B[1] * Visit[j] + B[2] * Age[i] + B[3] * MMSE[i] + B[4] * EDUC[i] + B[5] * Sex[i] + \
                        B[6] * eTIV[i] + B[7] * ASF[i] + B[8] * nWBV[i] + b[i]
            p = scipy.special.expit(eta[i, j])
            Y[i, j] = np.random.binomial(1, p)

    np.random.seed(1)

    mat = np.empty((N * m, 10))
    p = 0
    for i in range(N):
        jk = 0
        for jj in range(p, p + m):
            mat[jj, 0] = i + 1
            mat[jj, 1] = Visit[jk]
            mat[jj, 2] = Age[i]
            mat[jj, 3] = MMSE[i]
            mat[jj, 4] = EDUC[i]
            mat[jj, 5] = Sex[i]
            mat[jj, 6] = eTIV[jk]
            mat[jj, 7] = ASF[jk]
            mat[jj, 8] = nWBV[jk]
            mat[jj, 9] = Y[i, jk]
            jk += 1
        p += m

    # Convert the `mat` array to a DataFrame
    df_mat = pd.DataFrame(mat, columns=['ID', 'Visit', 'Age', 'MMSE', 'EDUC', 'Sex', 'eTIV', 'ASF', 'nWBV', 'Y'])
    df = pd.get_dummies(df_mat, columns=["Visit"], prefix="Visit")
    RANDOM_STATE = 13
    splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state=RANDOM_STATE)
    split = splitter.split(df, groups=df['ID'])
    train_indexes, test_indexes = next(split)

    X_train = df.iloc[train_indexes]
    X_test = df.iloc[test_indexes]
    age_scaler = MinMaxScaler()
    educ_scaler = MinMaxScaler()
    mmse_scaler = MinMaxScaler()
    etiv_scaler = MinMaxScaler()

    age_scaler.fit(X_train["Age"].to_numpy().reshape(-1, 1))
    educ_scaler.fit(X_train["EDUC"].to_numpy().reshape(-1, 1))
    mmse_scaler.fit(X_train["MMSE"].to_numpy().reshape(-1, 1))
    etiv_scaler.fit(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_train["Age"] = age_scaler.transform(X_train["Age"].to_numpy().reshape(-1, 1))
    X_train["EDUC"] = educ_scaler.transform(X_train["EDUC"].to_numpy().reshape(-1, 1))
    X_train["MMSE"] = mmse_scaler.transform(X_train["MMSE"].to_numpy().reshape(-1, 1))
    X_train["eTIV"] = etiv_scaler.transform(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_test["Age"] = age_scaler.transform(X_test["Age"].to_numpy().reshape(-1, 1))
    X_test["EDUC"] = educ_scaler.transform(X_test["EDUC"].to_numpy().reshape(-1, 1))
    X_test["MMSE"] = mmse_scaler.transform(X_test["MMSE"].to_numpy().reshape(-1, 1))
    X_test["eTIV"] = mmse_scaler.transform(X_test["eTIV"].to_numpy().reshape(-1, 1))
    visit_3 = df[df["Visit_4.0"] == 1][["ID", "Y"]]
    y_train_final = []
    for id in X_train["ID"]:
        y_train_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_test_final = []
    for id in X_test["ID"]:
        y_test_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_train_super_final = pd.DataFrame({
    "Group": y_train_final})
    y_test_super_final = pd.DataFrame({
    "Group": y_test_final})
    X_test_final = X_test.drop(["ID"], axis = 1)
    X_train_final = X_train.drop(["ID"], axis = 1)
    rows_n = 5

    X_train_super_final = np.reshape(X_train_final.to_numpy(),(X_train_final.shape[0]//5,X_train_final.shape[1],rows_n))
    X_test_super_final = np.reshape(X_test_final.to_numpy(),(X_test_final.shape[0]//5,X_train_final.shape[1],rows_n))
    lstm_rnn = build_lstm_rnn((X_train_super_final.shape[1],X_train_super_final.shape[2]) )
    lstm_rnn.fit(X_train_super_final, y_train_super_final, epochs = 100)
    not_final_lstm_rnn_prediction = lstm_rnn.predict(X_test_super_final)
    lstm_rnn_prediction = np.where(not_final_lstm_rnn_prediction > 0.5, 1, 0)
    lstm_rnn_conf = confusion_matrix(y_test_super_final,lstm_rnn_prediction)
    lstm_rnn_plot_conf = ConfusionMatrixDisplay(lstm_rnn_conf)
    lstm_rnn_plot_conf.plot()
    TP=lstm_rnn_conf[1,1]
    FP=lstm_rnn_conf[0,1]
    TN=lstm_rnn_conf[0,0]
    FN=lstm_rnn_conf[1,0]
    accuracy=(TP+TN)/(TP+FP+FN+TN)
    recall =TP/(TP+FN)
    precision =TP/(TP+TN)
    accuracy_list.append(accuracy)
    recall_list.append(recall)
    precision_list.append(precision)

print("Accuracy List:", accuracy_list)
print("Recall List:", recall_list)
print("Precision List:", precision_list)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13284\2922596249.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["Age"] = age_scaler.transform(X_train["Age"].to_numpy().reshape(-1, 1))
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13284\2922596249.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["EDUC"] = educ_scaler.transform(X_train["EDUC"].to_numpy().reshape(-1, 1))
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13284\2922596249.py:66: SettingWithCopyWarning: 
A value is trying to be s

ValueError: Data cardinality is ambiguous:
  x sizes: 80
  y sizes: 400
Make sure all arrays contain the same number of samples.